#### Set the environment variable

In [4]:
!export FUEL_DATA_PATH="/work/ajaynagesh/LadderNetworks/ladder/data"
# note: this is not setting the environ variable

In [3]:
import sys,os,os.path

In [4]:
os.environ['FUEL_DATA_PATH']="/work/ajaynagesh/LadderNetworks/ladder/data"

Hurray! The above is working ... 
see : https://stackoverflow.com/questions/37890898/how-to-set-env-variable-in-jupyter-notebook

#### Import MNIST

In [1]:
from fuel.datasets import MNIST

Using gpu device 0: Tesla P4 (CNMeM is disabled, cuDNN not available)


#### Experiments with the MNIST dataset

In [2]:
dataset_class = MNIST

In [3]:
??dataset_class

In [8]:
train_mnist = dataset_class(['train'])
test_mnist = dataset_class(['test'])

In [9]:
print( train_mnist.num_examples)
print( test_mnist.num_examples)

60000
10000


In [10]:
train_mnist.axis_labels

{u'features': (u'batch', u'channel', u'height', u'width'),
 u'targets': (u'batch', u'index')}

In [11]:
train_mnist.data_sources

(array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],
 
 
        [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],
 
 
        [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],
 
 
        ..., 
        [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., .

### Experiments by loading the blocks module of ladder

In [18]:
from run_emboot import *

##### get the logger output in IPython

https://stackoverflow.com/questions/18786912/get-output-from-the-logging-module-in-ipython-notebook

In [7]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


### Load the params from the saved params from the trained runs

With the following set of cmds I am able to load the saved params of a trained run


In [21]:
cli_params = dict()
cli_params['load_from'] = "/work/ajaynagesh/LadderNetworks/ladder/results/emboot_100_full0"

In [44]:
cli_params['data_type'] = 'test'
cli_params['no_load'] = 'g_'

Load the params and set up the data

In [25]:
p, loaded = load_and_log_params(cli_params)
_, data, whiten, cnorm = setup_data(p, test_set=True)

In [30]:
print len(data['test_ind'])
print len(data['train_ind'])
print len(data['valid_ind'])

916
12900
100


In [33]:
p.keys()

['load_from',
 'save_dir',
 'zestbn',
 'dseed',
 'commit',
 'top_c',
 'act',
 'batch_size',
 'dataset',
 'valid_set_size',
 'num_epochs',
 'whiten_zca',
 'unlabeled_samples',
 'decoder_spec',
 'denoising_cost_x',
 'f_local_noise_std',
 'cmd',
 'lrate_decay',
 'seed',
 'lr',
 'save_to',
 'super_noise_std',
 'valid_batch_size',
 'contrast_norm',
 'encoder_layers',
 'labeled_samples']

In [35]:
p.data_type = 'train'

In [38]:
ladder = setup_model(p)
dset, indices, calc_batchnorm = {
        'train': (data.train, data.train_ind, True), ## Changing from False
        'valid': (data.valid, data.valid_ind, True),
        'test':  (data.test, data.test_ind, True),
    }[p.data_type]

In [41]:
ds = make_datastream(dset, indices,
                         batch_size=p.get('batch_size'),
                         n_labeled=len(indices),
                         n_unlabeled=len(indices),
                         balanced_classes=False,
                         whiten=whiten,
                         cnorm=cnorm,
                         scheme=SequentialScheme)

In [42]:
ds.get_data

<bound method SemiDataStream.get_data of <run_emboot.SemiDataStream object at 0x7f2b46666950>>

In [48]:
cli_params['data_type'] = 'train'
targets, acts = analyze(cli_params)

INFO:main:== COMMAND LINE ==
INFO:main:/net/kate/storage/work/ajaynagesh/software/miniconda3/envs/ladder/lib/python2.7/site-packages/ipykernel/__main__.py -f /run/user/21964/jupyter/kernel-c13c0f0e-1b3d-45bb-be8b-b8ba7d053d03.json
INFO:main:== PARAMETERS ==
INFO:main: load_from           : None                 <- /work/ajaynagesh/LadderNetworks/ladder/results/emboot_100_full0
INFO:main: save_dir            : results/emboot_100_full_crtparams20 
INFO:main: zestbn              : bugfix               
INFO:main: dseed               : 1                    
INFO:main: commit              : f0efd0a55a2c419b332c8f730f86bcd5e4330be9 
INFO:main: top_c               : 1                    
INFO:main: act                 : relu                 
INFO:main: batch_size          : 100                  
INFO:main: dataset             : emboot               
INFO:main: data_type           : None                 <- train
INFO:main: valid_set_size      : 10000                
INFO:main: num_epochs       

data.train_ind [ 209 9577 5670 ..., 5373 2944 9987]
data.train_ind (12900,)
data - keys ['valid_ind', 'test_ind', 'train_ind', 'train', 'test', 'valid']
valid_ind --> SZ: 100
test_ind --> SZ: 916
train_ind --> SZ: 12900
train 13000
test 916
valid 13000


DEBUG:blocks.monitoring.evaluators:variable to evaluate: f_1_bn_mean_clean
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for f_1_bn_mean_clean
DEBUG:blocks.monitoring.evaluators:variable to evaluate: f_1_bn_var_clean
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for f_1_bn_var_clean
DEBUG:blocks.monitoring.evaluators:variable to evaluate: f_2_bn_mean_clean
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for f_2_bn_mean_clean
DEBUG:blocks.monitoring.evaluators:variable to evaluate: f_2_bn_var_clean
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for f_2_bn_var_clean
DEBUG:blocks.monitoring.evaluators:variable to evaluate: f_3_bn_mean_clean
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for f_3_bn_mean_clean
DEBUG:blocks.monito

e 0, i 0:VF_C_class 0.815, VF_E 26, VF_C_de 0.00126 0.623 0.981 0.76


##### Reload run_emboot.py with the changed file and also print the batch size

However to effective reload , we need to restart the kernel 
See: https://support.enthought.com/hc/en-us/articles/204469240-Jupyter-IPython-After-editing-a-module-changes-are-not-effective-without-kernel-restart

In [5]:
import sys,os,os.path
os.environ['FUEL_DATA_PATH']="/work/ajaynagesh/LadderNetworks/ladder/data"

cli_params = dict()
cli_params['load_from'] = "/work/ajaynagesh/LadderNetworks/ladder/results/emboot_100_full0"
cli_params['no_load'] = 'g_'
cli_params['data_type'] = 'train'

In [6]:
from run_emboot import *
targets, acts = analyze(cli_params)

------------------------------------------ Batch Sz = 100 ---------
data.train_ind [ 209 9577 5670 ..., 5373 2944 9987]
data.train_ind (12900,)
data - keys ['valid_ind', 'test_ind', 'train_ind', 'train', 'test', 'valid']
valid_ind --> SZ: 100
test_ind --> SZ: 916
train_ind --> SZ: 12900
train 13000
test 916
valid 13000
e 0, i 0:VF_C_class 0.815, VF_E 26, VF_C_de 0.00126 0.623 0.981 0.76


In [8]:
guess = numpy.argmax(acts, axis=1)
correct = numpy.sum(numpy.equal(guess, targets.flatten()))

### Error on the train dataset

In [9]:
print (1. - correct / float(len(guess))) * 100.

23.4573643411


### EMBOOT Dataset

In [10]:
from fuel_converter import EMBOOT

In [13]:
emboot_train = EMBOOT(['train'])
emboot_test = EMBOOT(['test'])

In [14]:
print emboot_train.num_examples
print emboot_test.num_examples

13000
916


In [18]:
print emboot_train.axis_labels

{u'features': (u'batch', u'feature'), u'targets': (u'batch', u'index')}


In [19]:
print emboot_train.data_sources

(array([[-0.03081121, -0.05515246, -0.00701181, ..., -0.03319645,
        -0.00103687, -0.03405993],
       [-0.00316652, -0.08585572,  0.00378593, ..., -0.0073941 ,
         0.04546531, -0.06978292],
       [ 0.04344497, -0.05054203,  0.00522156, ...,  0.04237889,
         0.01932781,  0.05354481],
       ..., 
       [-0.01100759,  0.00742121,  0.04095598, ...,  0.02451436,
         0.03807044,  0.00670465],
       [ 0.02293569, -0.1489239 ,  0.05471994, ...,  0.04448456,
         0.06721334,  0.02222161],
       [ 0.07164202,  0.02218784,  0.06132978, ...,  0.01108115,
         0.07298984,  0.05412074]]), array([[1],
       [1],
       [2],
       ..., 
       [2],
       [0],
       [0]], dtype=uint8))


In [21]:
from fuel.datasets import MNIST
mnist_train = MNIST(['train'])

In [22]:
mnist_train.data_sources

(array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],
 
 
        [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],
 
 
        [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],
 
 
        ..., 
        [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., .

## Load the ladder model trained on MNIST

using 100 examples for supervision

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

import sys,os,os.path
os.environ['FUEL_DATA_PATH']="/work/ajaynagesh/LadderNetworks/ladder/data"

cli_params = dict()
cli_params['load_from'] = "/work/ajaynagesh/LadderNetworks/ladder.orig/results/mnist_100_full0"
cli_params['no_load'] = 'g_'
cli_params['data_type'] = 'test'

from run import *
targets, acts = analyze(cli_params)

guess = numpy.argmax(acts, axis=1)
correct = numpy.sum(numpy.equal(guess, targets.flatten()))

DEBUG:root:test
INFO:main:== COMMAND LINE ==
INFO:main:/net/kate/storage/work/ajaynagesh/software/miniconda3/envs/ladder/lib/python2.7/site-packages/ipykernel/__main__.py -f /run/user/21964/jupyter/kernel-c13c0f0e-1b3d-45bb-be8b-b8ba7d053d03.json
INFO:main:== PARAMETERS ==
INFO:main: load_from           : None                 <- /work/ajaynagesh/LadderNetworks/ladder.orig/results/mnist_100_full0
INFO:main: save_dir            : results/mnist_100_full0 
INFO:main: zestbn              : bugfix               
INFO:main: dseed               : 1                    
INFO:main: commit              : 5a8daa1760535ec4aa25c20c531e1cc31c76d911 
INFO:main: top_c               : 1                    
INFO:main: act                 : relu                 
INFO:main: batch_size          : 100                  
INFO:main: dataset             : mnist                
INFO:main: data_type           : None                 <- test
INFO:main: valid_set_size      : 10000                
INFO:main: num_epochs

DEBUG:blocks.monitoring.evaluators:variable to evaluate: denois3
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for denois3
DEBUG:blocks.monitoring.evaluators:variable to evaluate: denois4
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for denois4
DEBUG:blocks.monitoring.evaluators:variable to evaluate: denois5
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for denois5
DEBUG:blocks.monitoring.evaluators:variable to evaluate: denois6
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for denois6
DEBUG:blocks.monitoring.evaluators:Compiling initialization and readout functions
DEBUG:blocks.monitoring.evaluators:Initialization and readout functions compiled
INFO:main.nn:Batch norm parameters: f_1_bn_mean_clean, f_1_bn_var_clean, f_2_bn_mean_clean, f_2_bn_var_clean, f_3_bn_mean_clean, 

e 0, i 0:VF_C_class nan, VF_E nan, VF_C_de nan nan nan nan


### Accuracy

In [3]:
print (1. - correct / float(len(guess))) * 100.

1.05


### New Emboot dataset with 80/20 split

In [1]:
from fuel_converter_new import EMBOOT_NEW
import sys,os,os.path
os.environ['FUEL_DATA_PATH']="/work/ajaynagesh/LadderNetworks/ladder/data"

Using gpu device 0: Tesla P4 (CNMeM is disabled, cuDNN not available)


In [2]:
emboot_new_train = EMBOOT_NEW(['train'])
emboot_new_test = EMBOOT_NEW(['test'])

In [3]:
print emboot_new_train.num_examples
print emboot_new_test.num_examples

10400
2600


## Emboot experiments with the 80/20 split

In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

import sys,os,os.path
os.environ['FUEL_DATA_PATH']="/work/ajaynagesh/LadderNetworks/ladder/data"

cli_params = dict()
cli_params['load_from'] = "/work/ajaynagesh/LadderNetworks/ladder/results/emboot_80_20split0"
cli_params['no_load'] = 'g_'
cli_params['data_type'] = 'test'

DEBUG:root:test


In [2]:
from run_emboot import *
targets, acts = analyze(cli_params)

guess = numpy.argmax(acts, axis=1)
correct = numpy.sum(numpy.equal(guess, targets.flatten()))

INFO:main:== COMMAND LINE ==
INFO:main:/net/kate/storage/work/ajaynagesh/software/miniconda3/envs/ladder/lib/python2.7/site-packages/ipykernel/__main__.py -f /run/user/21964/jupyter/kernel-c13c0f0e-1b3d-45bb-be8b-b8ba7d053d03.json
INFO:main:== PARAMETERS ==
INFO:main: load_from           : None                 <- /work/ajaynagesh/LadderNetworks/ladder/results/emboot_80_20split0
INFO:main: save_dir            : results/emboot_80_20split0 
INFO:main: zestbn              : bugfix               
INFO:main: dseed               : 1                    
INFO:main: commit              : bef5430559b8b0450f95944e60ca648b657686da 
INFO:main: top_c               : 1                    
INFO:main: act                 : relu                 
INFO:main: batch_size          : 100                  
INFO:main: dataset             : emboot               
INFO:main: data_type           : None                 <- test
INFO:main: valid_set_size      : 10000                
INFO:main: num_epochs          : 150

------------------------------------------ Batch Sz = 100 ---------
data.train_ind [8364 7892 3583 ...,  905 5192  235]
data.train_ind (10400,)
data - keys ['valid_ind', 'test_ind', 'train_ind', 'train', 'test', 'valid']
valid_ind --> SZ: 0
test_ind --> SZ: 2600
train_ind --> SZ: 10400
train 10400
test 2600
valid 10400


DEBUG:blocks.monitoring.evaluators:variable to evaluate: f_1_bn_mean_clean
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for f_1_bn_mean_clean
DEBUG:blocks.monitoring.evaluators:variable to evaluate: f_1_bn_var_clean
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for f_1_bn_var_clean
DEBUG:blocks.monitoring.evaluators:variable to evaluate: f_2_bn_mean_clean
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for f_2_bn_mean_clean
DEBUG:blocks.monitoring.evaluators:variable to evaluate: f_2_bn_var_clean
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for f_2_bn_var_clean
DEBUG:blocks.monitoring.evaluators:variable to evaluate: f_3_bn_mean_clean
DEBUG:blocks.monitoring.evaluators:Using the default  (average over minibatches) aggregation scheme for f_3_bn_mean_clean
DEBUG:blocks.monito

e 0, i 0:VF_C_class nan, VF_E nan, VF_C_de nan nan nan nan


### Accuracy

This is the test error on Emboot test data

Training dataset size: 10400
Test dataset size: 2600

80/20 split

In [4]:
print (1. - correct / float(len(guess))) * 100.

21.9230769231


##### Error on test = 21.9230769231
##### Error on train = 22.663462

#### Using validation (400 data points) during training

#### Error on Test = 22.769231
#### Error on Train = 23.050000

### Using only 20 datapoints as supervision

directory: results/emboot_80_20_only20_1

cmds: 

THEANO_FLAGS=device=gpu2  python run_emboot.py train --encoder-layers 500-250-250-250-4 --decoder-spec gauss --denoising-cost-x 1000,10,0.1,0.1,0.1,0.1 --labeled-samples 20 --unlabeled-samples 10400 --seed 1 --dataset emboot  -- emboot_80_20_only20_

 python ./run_emboot.py evaluate  results/emboot_80_20_only20_1

#### test error: 51.730769
#### train error: 29.048077

### Using only 40 datapoints as supervision

directory: emboot_80_20_only40_0

cmds: 


THEANO_FLAGS=device=gpu2  python run_emboot.py train --encoder-layers 500-250-250-250-4 --decoder-spec gauss --denoising-cost-x 1000,10,0.1,0.1,0.1,0.1 --labeled-samples 40 --unlabeled-samples 10400 --seed 1 --dataset emboot  -- emboot_80_20_only40_


python ./run_emboot.py evaluate  results/emboot_80_20_only40_0

#### test error: 26.923077
#### train error: 24.250000

### Using only 200 datapoints as supervision

directory: emboot_80_20_only200_0



#### test error:  19.807692
#### train error: 20.894231

### Using only 400 datapoints as supervision

directory: emboot_80_20_only400_0

#### test error:  18.076923
#### train error: 18.942308

### Using only 800 datapoints as supervision

directory: emboot_80_20_only800_0

#### test error:  17.692308
#### train error: 17.951923